In [ ]:
#Clean The Data
import csv
import os
import pandas as pd
import numpy as np
import functools
#Join Conditions
def conjunction(*conditions):
    return functools.reduce(np.logical_and, conditions)
#Kelvin to Fahrenheit conversion
def kelvin_to_fahrenheit(tempereture_kelvin):
   tempereture_fahrenheit = (((tempereture_kelvin - 273) * 9/5) + 32)
   return tempereture_fahrenheit

os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Team-Folder/')

Trip_Data = pd.read_csv("fhv_tripdata_2018-01.csv", parse_dates=[0], low_memory=False)
Weather_Data = pd.read_csv('NYC_Weather.csv', parse_dates=[0], low_memory=False)
Zone_Data = pd.read_csv('fhv_zone_lookup.csv', parse_dates=[0], low_memory=False)
Zip_Data = pd.read_csv("fhv_zone_lookup_zipcode.csv", low_memory=False)

Trip_Data = Trip_Data.fillna(0)

Trip_Data['Pickup_Date'] = Trip_Data.Pickup_DateTime.dt.date

Trip_Data = Trip_Data.groupby('Pickup_Date').apply(pd.DataFrame.sample, n=10000, replace=False).reset_index(drop=True) #Get 10,000 random rows from each date

is_not_0_p =  Trip_Data['PUlocationID']!=0 #no location 
is_not_264_p =  Trip_Data['PUlocationID']!=264 #unknown location
is_not_265_p =  Trip_Data['PUlocationID']!=265 #unknown location

is_not_0_d =  Trip_Data['DOlocationID']!=0 #no location 
is_not_264_d =  Trip_Data['DOlocationID']!=264 #unknown location
is_not_265_d =  Trip_Data['DOlocationID']!=265 #unknown location

Trip_Data = Trip_Data[conjunction(is_not_0_p,is_not_264_p,is_not_265_p,is_not_0_d,is_not_264_d,is_not_265_d)] #clean data

Trip_Data['Pickup_DateTime'] = pd.to_datetime(Trip_Data['Pickup_DateTime'])
Trip_Data['DropOff_datetime'] = pd.to_datetime(Trip_Data['DropOff_datetime'])
Trip_Data['PUlocationID'] = pd.to_numeric(Trip_Data['PUlocationID'])
Trip_Data['DOlocationID'] = pd.to_numeric(Trip_Data['DOlocationID'])

Trip_Data['Pickup_DateTime2'] = Trip_Data.Pickup_DateTime.dt.round('H')

Weather_Data['dt_iso'] = pd.to_datetime(Weather_Data['dt_iso'])
Weather_Data = Weather_Data.fillna(0)

Zone_Data['LocationID'] = pd.to_numeric(Zone_Data['LocationID'])
Zone_Data = Zone_Data.fillna(0)

Combined_Data = pd.merge(Trip_Data, Weather_Data, how='left', left_on=['Pickup_DateTime2'], right_on=['dt_iso'])

Combined_Data = pd.merge(Combined_Data, Zone_Data, how='left', left_on=['PUlocationID'], right_on=['LocationID'])

Combined_Data = pd.merge(Combined_Data, Zone_Data, how='left', left_on=['DOlocationID'], right_on=['LocationID'])

Combined_Data = Combined_Data[['Pickup_DateTime', 'DropOff_datetime', 'PUlocationID', 'DOlocationID', 'SR_Flag', 'city_name', 'temp', 'humidity', 'wind_speed', 'weather_id', 'weather_main', 'weather_description', 'Borough_x', 'Zone_x', 'Borough_y', 'Zone_y']]

Combined_Data_Fixed = Combined_Data.rename(columns={'Borough_x': 'Pickup_Location', 'Zone_x': 'Pickup_Neighborhood', 'Borough_y': 'Dropoff_Location', 'Zone_y': 'Dropoff_Neighborhood'})

Combined_Data_Fixed.drop_duplicates(subset=None, keep='first', inplace=False)

Combined_Data_Fixed['Trip_Duration'] = ( pd.to_timedelta(Combined_Data_Fixed.DropOff_datetime).dt.total_seconds() - pd.to_timedelta(Combined_Data_Fixed.Pickup_DateTime).dt.total_seconds() ) / 60

Combined_Data_Fixed = pd.merge(Combined_Data_Fixed, Zip_Data, how='left', left_on=['PUlocationID'], right_on=['LocationID'])

Combined_Data_Fixed = pd.merge(Combined_Data_Fixed, Zip_Data, how='left', left_on=['DOlocationID'], right_on=['LocationID'])

Combined_Data_Fixed = Combined_Data_Fixed.rename(columns={'zipcode_x': 'Pickup_ZipCode', 'zipcode_y': 'Dropoff_ZipCode'})

Combined_Data_Fixed['Temp'] = kelvin_to_fahrenheit(Combined_Data_Fixed['temp']).astype(int)

Combined_Data_Fixed['Pickup_City'] = Combined_Data_Fixed["Pickup_Neighborhood"]+', '+Combined_Data_Fixed["Pickup_Location"]

Combined_Data_Fixed['Dropoff_City'] = Combined_Data_Fixed["Dropoff_Neighborhood"]+', '+Combined_Data_Fixed["Dropoff_Location"]

Combined_Data_Fixed = Combined_Data_Fixed[['Pickup_DateTime', 'DropOff_datetime', 'PUlocationID', 'DOlocationID', 'SR_Flag', 'Pickup_City', 'Dropoff_City', 'Pickup_ZipCode', 'Dropoff_ZipCode', 'Trip_Duration', 'Temp', 'humidity', 'wind_speed', 'weather_id', 'weather_main', 'weather_description']]

Combined_Data_Fixed.to_csv("RideShare_Data_201801.csv", index=False)

print('done')
